# Knowledge Mapper

To create a embedding visualization based on text data from Unit handbook that helps students decide which courses to pursue based on their interests and current enrolled units.

```
Key fields:
1. UNIT_TITLE
2. HANDBOOK_SYNOPSIS
3. UNIT_LEARNING_OUTCOME

Unit meta data:
1. UNIT_CODE
2. ABBREVIATED_UNIT_TITLE
Colour by:
1. STUDY_LEVEL (UG vs. PG)
2. OWNING_FACULTY (faculty who teaches it)
3. OWNING_ORG_UNIT (department who teaches it)

Auxillary questions:
1. What other units should be prohibited to be taken in dual with this unit?
2. What units should I take before this unit? 

Note: PUBLISH_TO_HANDBOOK probably should be used as a first-order filter.
```

### [Embedding Projector](https://projector.tensorflow.org/)

Data format:
```
Load data from your computer
Step 1: Load a TSV file of vectors.
Example of 3 vectors with dimension 4:
0.1\t0.2\t0.5\t0.9
0.2\t0.1\t5.0\t0.2
0.4\t0.1\t7.0\t0.8
Step 2 (optional): Load a TSV file of metadata.
Example of 3 data points and 2 columns.
Note: If there is more than one column, the first row will be parsed as column labels.
Pokémon\tSpecies
Wartortle\tTurtle
Venusaur\tSeed
Charmeleon\tFlame
```

Metadata
```
UNIT_CODE\tUNIT_TITLE\tHANDBOOK_SYNOPSIS\tUNIT_LEARNING_OUTCOME\tSTUDY_LEVEL\tOWNING_FACULTY\tOWNING_ORG_UNIT
```

### Libraries

In [53]:
!pip install transformers

     |████████████████████████████████| 1.8MB 5.7MB/s 
     |████████████████████████████████| 3.2MB 17.9MB/s 
     |████████████████████████████████| 890kB 40.9MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893261 sha256=11b0b2132a9cd580db57ec69d847cefdffa4586f0751f3807069bbd177814e4c
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [ ]:
!pip install sentence-transformers

     |████████████████████████████████| 71kB 3.4MB/s 
     |████████████████████████████████| 1.2MB 5.5MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-0.4.1.2-cp36-none-any.whl size=103068 sha256=a09c3dea0083f93d652fb4acb1de2cbf48f6ba24338896f5548c3a8b84b7c4da
  Stored in directory: /root/.cache/pip/wheels/3d/33/d1/5703dd56199c09d4a1b41e0c07fb4e7765a84d787cbdc48ac3
Successfully built sentence-transformers


In [3]:
# general
import json
import numpy as np
import pandas as pd
from pathlib import Path
from sentence_transformers import SentenceTransformer

In [86]:
data_path = Path('/content/drive/MyDrive/knowledge-mapper/data/raw_data/units owned by faculty - one line per unit (as at 18 Feb 2021).xlsx')
df = pd.read_excel(data_path, sheet_name='Sheet1')

In [87]:
df.shape

(5848, 36)

In [88]:
df.columns

Index(['CL_UNIT_ID', 'UNIT_CODE', 'CL_UNIT_VERSION', 'UNIT_TITLE',
       'ABBREVIATED_UNIT_TITLE', 'CREDIT_POINTS', 'UNIT_STATUS',
       'OWNING_FACULTY', 'OWNING_ORG_UNIT', 'HIGHEST_SCA_BAND',
       'IMPLEMENTATION_YR', 'PUBLISH_TO_HANDBOOK', 'STUDY_LEVEL',
       'HIGHEST_SCA_BAND_1', 'UNIT_EFTSL', 'HANDBOOK_SYNOPSIS',
       'WORKLOAD_REQUIREMENTS', 'QUOTA_INFORMATION', 'OTHER_UNIT_COSTS',
       'FIELD_WORK', 'AREA_OF_STUDY_LINKS', 'OFF_CAMPUS_ATTEND_REQUIREMENTS',
       'SPECIAL_NOTE_TO_STUDENTS', 'UNITOFFERING',
       'HANDBOOK_ASSESSMENT_SUMMARY', 'ASSES_ITEMS', 'UNITCOORD', 'CHIEFEXAM',
       'TEACHING_RESPONSIBILITY', 'PREREQUISITE', 'COREQUISITE', 'PROHIBITION',
       'RULES(PREREQ,COREQ,PROH)', 'INFORMATION_RULE', 'LEARNING_OUTCOME_INFO',
       'UNIT_LEARNING_OUTCOME'],
      dtype='object')

In [89]:
df.head()

,CL_UNIT_ID,UNIT_CODE,CL_UNIT_VERSION,UNIT_TITLE,ABBREVIATED_UNIT_TITLE,CREDIT_POINTS,UNIT_STATUS,OWNING_FACULTY,OWNING_ORG_UNIT,HIGHEST_SCA_BAND,IMPLEMENTATION_YR,PUBLISH_TO_HANDBOOK,STUDY_LEVEL,HIGHEST_SCA_BAND_1,UNIT_EFTSL,HANDBOOK_SYNOPSIS,WORKLOAD_REQUIREMENTS,QUOTA_INFORMATION,OTHER_UNIT_COSTS,FIELD_WORK,AREA_OF_STUDY_LINKS,OFF_CAMPUS_ATTEND_REQUIREMENTS,SPECIAL_NOTE_TO_STUDENTS,UNITOFFERING,HANDBOOK_ASSESSMENT_SUMMARY,ASSES_ITEMS,UNITCOORD,CHIEFEXAM,TEACHING_RESPONSIBILITY,PREREQUISITE,COREQUISITE,PROHIBITION,"RULES(PREREQ,COREQ,PROH)",INFORMATION_RULE,LEARNING_OUTCOME_INFO,UNIT_LEARNING_OUTCOME
0,554c86a41b5aac10653b206b274bcbf9,ACB1020,2021.04RO,Accounting in business,ACC IN BUS,6.0,Accredited,Faculty of Business and Economics,Department of Accounting,SCA Band 4,2021,Y,undergraduate,SCA Band 4,0.125,This unit introduces basic accounting concepts...,Minimum total expected workload to achieve the...,NaN,NaN,NaN,NaN,NaN,NaN,S1-01-PENINSULA-ON-CAMPUS Offered-Y,NaN,1 - 50% APPLY_TO_ALL_OFFER - Y\n2 - 50% APPL...,NaN,Dr Mahendra Goyal,Responsible teaching Department of Accounting ...,NaN,NaN,ACB1120 OR ACF1100 OR ACF1200 OR ACC1100 OR AC...,Prohibition: ACB1120 OR ACF1100 OR ACF1200 OR ...,NaN,"On successful completion of this unit, you sho...",ULO1 - demonstrate an understanding of various...
1,1928deab1bd65c504c45bbbbdc4bcb3a,ACB1100,2021.01RO,Introduction to financial accounting,INTRO FIN ACC,6.0,Accredited,Faculty of Business and Economics,Department of Accounting,SCA Band 3,2021,N,undergraduate,SCA Band 3,0.125,This unit provides students with an introducti...,Minimum total expected workload to achieve the...,NaN,Costs are indicative and subject to change.Ele...,NaN,NaN,NaN,NaN,NaN,Within semester assessment: 50% + Examination:...,NaN,NaN,Dr Mahendra Goyal,Responsible teaching Department of Accounting ...,NaN,NaN,ACC1100 OR ACF1100 OR ACW1100,Prohibition: ACC1100 OR ACF1100 OR ACW1100,NaN,The learning outcomes associated with this uni...,ULO1 - Identify and analyse measurement system...
2,e14cc6a41b5aac10653b206b274bcb3e,ACB1120,2021.04RO,Financial accounting 1,INTRO FIN ACC,6.0,Accredited,Faculty of Business and Economics,Department of Accounting,SCA Band 4,2021,Y,undergraduate,SCA Band 4,0.125,This unit provides you with an introduction to...,Minimum total expected workload to achieve the...,NaN,NaN,NaN,NaN,NaN,NaN,S1-01-PENINSULA-ON-CAMPUS Offered-Y,NaN,1 - 50% APPLY_TO_ALL_OFFER - Y\n2 - 50% APPL...,NaN,Dr Mahendra Goyal,Responsible teaching Department of Accounting ...,NaN,NaN,ACC1100 OR ACF1100 OR ACW1120,Prohibition: ACC1100 OR ACF1100 OR ACW1120,NaN,"On successful completion of this unit, you sho...",ULO1 - identify and analyse measurement system...
3,a128deab1bd65c504c45bbbbdc4bcbb4,ACB1200,2021.01RO,Accounting for managers,ACC FOR MNGRS,6.0,Accredited,Faculty of Business and Economics,Department of Accounting,SCA Band 3,2021,N,undergraduate,SCA Band 3,0.125,This unit introduces basic accounting concepts...,Minimum total expected workload to achieve the...,NaN,Costs are indicative and subject to change.Ele...,NaN,NaN,NaN,NaN,NaN,Within semester assessment: 50% + Examination:...,NaN,NaN,Mr Jonathan Phillips,Responsible teaching Department of Accounting ...,NaN,NaN,ACF1100 OR ACW1100 OR ACB1100 OR ACC1200 OR AC...,Prohibition: ACF1100 OR ACW1100 OR ACB1100 OR ...,NaN,The learning outcomes associated with this uni...,ULO1 - Ddemonstrate an understanding of variou...
4,b528deab1bd65c504c45bbbbdc4bcbc7,ACB2020,2021.01RO,Cost information for decision making,COST INFO FOR DEC MA,6.0,Accredited,Faculty of Business and Economics,Department of Accounting,SCA Band 3,2021,N,undergraduate,SCA Band 3,0.125,Introduction to management accounting. Topics ...,Minimum total expected workload to achieve the...,NaN,Costs are indicative and subject to change.Ele...,NaN,NaN,NaN,NaN,NaN,Within semester assessment: 50% + Examination:...,NaN,NaN,Mr Paul Yap,Responsible teaching Department of Accounting ...,ACB1100,NaN,NaN,Prerequisite:

### Unique Identifiers

In [90]:
df.CL_UNIT_ID.nunique(), df.UNIT_CODE.nunique()

(5848, 5848)

In [91]:
key_cols = ["CL_UNIT_ID",
            "UNIT_CODE",
            "UNIT_TITLE",
            "HANDBOOK_SYNOPSIS",
            "UNIT_LEARNING_OUTCOME",
            "STUDY_LEVEL",
            "OWNING_FACULTY",
            "OWNING_ORG_UNIT"
            ]

### Filter by `PUBLISH_TO_HANDBOOK`

In [92]:
df = df.loc[df["PUBLISH_TO_HANDBOOK"] == 'Y', :]

In [93]:
df.shape

(5324, 36)

In [94]:
df.head()

,CL_UNIT_ID,UNIT_CODE,CL_UNIT_VERSION,UNIT_TITLE,ABBREVIATED_UNIT_TITLE,CREDIT_POINTS,UNIT_STATUS,OWNING_FACULTY,OWNING_ORG_UNIT,HIGHEST_SCA_BAND,IMPLEMENTATION_YR,PUBLISH_TO_HANDBOOK,STUDY_LEVEL,HIGHEST_SCA_BAND_1,UNIT_EFTSL,HANDBOOK_SYNOPSIS,WORKLOAD_REQUIREMENTS,QUOTA_INFORMATION,OTHER_UNIT_COSTS,FIELD_WORK,AREA_OF_STUDY_LINKS,OFF_CAMPUS_ATTEND_REQUIREMENTS,SPECIAL_NOTE_TO_STUDENTS,UNITOFFERING,HANDBOOK_ASSESSMENT_SUMMARY,ASSES_ITEMS,UNITCOORD,CHIEFEXAM,TEACHING_RESPONSIBILITY,PREREQUISITE,COREQUISITE,PROHIBITION,"RULES(PREREQ,COREQ,PROH)",INFORMATION_RULE,LEARNING_OUTCOME_INFO,UNIT_LEARNING_OUTCOME
0,554c86a41b5aac10653b206b274bcbf9,ACB1020,2021.04RO,Accounting in business,ACC IN BUS,6.0,Accredited,Faculty of Business and Economics,Department of Accounting,SCA Band 4,2021,Y,undergraduate,SCA Band 4,0.125,This unit introduces basic accounting concepts...,Minimum total expected workload to achieve the...,NaN,NaN,NaN,NaN,NaN,NaN,S1-01-PENINSULA-ON-CAMPUS Offered-Y,NaN,1 - 50% APPLY_TO_ALL_OFFER - Y\n2 - 50% APPL...,NaN,Dr Mahendra Goyal,Responsible teaching Department of Accounting ...,NaN,NaN,ACB1120 OR ACF1100 OR ACF1200 OR ACC1100 OR AC...,Prohibition: ACB1120 OR ACF1100 OR ACF1200 OR ...,NaN,"On successful completion of this unit, you sho...",ULO1 - demonstrate an understanding of various...
2,e14cc6a41b5aac10653b206b274bcb3e,ACB1120,2021.04RO,Financial accounting 1,INTRO FIN ACC,6.0,Accredited,Faculty of Business and Economics,Department of Accounting,SCA Band 4,2021,Y,undergraduate,SCA Band 4,0.125,This unit provides you with an introduction to...,Minimum total expected workload to achieve the...,NaN,NaN,NaN,NaN,NaN,NaN,S1-01-PENINSULA-ON-CAMPUS Offered-Y,NaN,1 - 50% APPLY_TO_ALL_OFFER - Y\n2 - 50% APPL...,NaN,Dr Mahendra Goyal,Responsible teaching Department of Accounting ...,NaN,NaN,ACC1100 OR ACF1100 OR ACW1120,Prohibition: ACC1100 OR ACF1100 OR ACW1120,NaN,"On successful completion of this unit, you sho...",ULO1 - identify and analyse measurement system...
5,0ded927adb5e68102bdd077cd39619c4,ACB2120,2021.05,Financial accounting 2,FIN ACCT 2,6.0,Accredited,Faculty of Business and Economics,Department of Accounting,SCA Band 4,2021,Y,undergraduate,SCA Band 4,0.125,This unit provides an overview of the current ...,Minimum total expected workload to achieve the...,NaN,NaN,NaN,NaN,NaN,NaN,S1-01-PENINSULA-ON-CAMPUS Offered-Y,�\n�,1 - 50% APPLY_TO_ALL_OFFER - Y\n2 - 50% APPL...,NaN,Dr Lisa Powell,Responsible teaching Department of Accounting ...,ACB1120 OR ACC1100 OR ACF1100 OR ACW1120,NaN,ACF2100 OR ACW2120 OR ACC2100,Prerequisite: ACB1120 OR ACC1100 OR ACF1100 OR...,NaN,"On successful completion of this unit, you sho...","ULO1 - explain the content of, and regulatory ..."
6,7d4cc6a41b5aac10653b206b274bcb9a,ACB2220,2021.03RO,Management accounting 1,MGT ACCT 1,6.0,Accredited,Faculty of Business and Economics,Department of Accounting,SCA Band 4,2021,Y,undergraduate,SCA Band 4,0.125,Introduction to management accounting. Topics ...,Minimum total expected workload to achieve the...,NaN,NaN,NaN,NaN,NaN,NaN,S1-01-PENINSULA-ON-CAMPUS Offered-Y,NaN,1 - 50% APPLY_TO_ALL_OFFER - Y\n2 - 50% APPL...,NaN,Dr John Ko,Responsible teaching Department of Accounting ...,ACC1100 OR ACF1100 OR ACW1120 OR ACB1120,NaN,ACC2200 OR ACF2200 OR ACW2220,Prerequisite: ACC1100 OR ACF1100 OR ACW1120 OR...,NaN,"On successful completion of this unit, you sho...",ULO1 - describe cost behaviour under different...
7,b14cc6a41b5aac10653b206b274bcbbd,ACB2420,2021.04RO,Accounting information systems,ACC INFO SYS,6.0,Accredited,Faculty of Business and Economics,Department of Accounting,SCA Band 4,2021,Y,undergraduate,SCA Band 4,0.125,"The objective of this unit is two-fold. First,...",Minimum total expected workload to achieve the...,NaN,NaN,NaN,NaN,NaN,NaN,S2-01-PENINSULA-ON-CAMPUS Offered-Y,NaN,1 - 50% APPLY_TO_ALL_OFFER - Y\n2 - 50% APPL...,NaN,Dr Daisy Seng,Responsible teaching Department of Accounting ...,ACB1020 OR ACF1100 OR ACB1120 OR ACF1200 OR AC...,NaN,

### Get key columns

In [99]:
df_txt = df.loc[:, key_cols]

In [100]:
df_txt.isna().sum()

CL_UNIT_ID                 0
UNIT_CODE                  0
UNIT_TITLE                 0
HANDBOOK_SYNOPSIS         27
UNIT_LEARNING_OUTCOME    378
STUDY_LEVEL                2
OWNING_FACULTY             0
OWNING_ORG_UNIT            0
dtype: int64

In [101]:
df_txt.fillna("", inplace=True)

In [102]:
df_txt.isna().sum()

CL_UNIT_ID               0
UNIT_CODE                0
UNIT_TITLE               0
HANDBOOK_SYNOPSIS        0
UNIT_LEARNING_OUTCOME    0
STUDY_LEVEL              0
OWNING_FACULTY           0
OWNING_ORG_UNIT          0
dtype: int64

In [56]:
model_name = 'distilbert-base-nli-stsb-mean-tokens'
model = SentenceTransformer(model_name) # load DistillBERT model (more efficient) 

100%|██████████| 245M/245M [00:34<00:00, 7.07MB/s]


In [107]:
def get_bert_embeddings(text, model):
    """Computes the mean BERT embeddings (context dependent) for a given sentence
    Returns a 768 dimensional embedding
    """
    if text.strip() != "":
        embeddings = model.encode(text)
    else:
        embeddings = np.zeros(model.get_sentence_embedding_dimension())
    return embeddings

In [78]:
df_txt.loc[0, 'HANDBOOK_SYNOPSIS'], df_txt.loc[0, 'UNIT_LEARNING_OUTCOME']

('This unit introduces basic accounting concepts to non-accountants. The information requirements of two main groups of information users are examined - external users such as current and potential investors and internal users such as managers. This unit provides an introduction to the structure, meaning, analysis and interpretation of financial statements, in addition to exploring financial issues confronting managers, such as cost and performance measurement and budgeting.',
 'ULO1 - demonstrate an understanding of various forms of business organisations\nULO2 - apply financial and management accounting principles in the preparation of financial statements\nULO3 - measure and interpret information relating to financial performance, financial position, liquidity and risk indicators of businesses\nULO4 - measure and interpret financial and non-financial information for managers to use in planning, decision making and control\nULO5 - develop the ability to work effectively in a team and

### Baseline

Few strategies:
- Combine embeddings (mean/sum) for both `HANDBOOK_SYNOPSIS` and `UNIT_LEARNING_OUTCOME`
- Keep them seperate to provide two distinct "modes" for viz that the users can select from
- Show them the top-K similar UNITS in terms of synopsis or outcomes

In [81]:
get_bert_embeddings(df_txt.loc[0, 'HANDBOOK_SYNOPSIS'], model).shape

(768,)

In [110]:
df_txt['embeddings'] = df_txt['HANDBOOK_SYNOPSIS'].apply(lambda x: get_bert_embeddings(x, model))

In [126]:
df_embeddings = pd.DataFrame(df_txt["embeddings"].to_list())

In [127]:
df_embeddings

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,728,729,730,731,732,733,734,735,736,737,738,739,740,741,742,743,744,745,746,747,748,749,750,751,752,753,754,755,756,757,758,759,760,761,762,763,764,765,766,767
0,-0.155681,0.489606,-0.421600,-0.072159,-0.067547,0.274299,0.091877,-0.698768,-0.666343,-0.536237,0.291072,0.899175,-0.077659,-0.613539,0.343758,-0.670124,0.794246,-0.388422,0.641546,-0.456669,-0.119552,0.153932,0.466036,0.399631,-0.440171,-0.322608,0.073568,-0.297683,-0.059697,-1.148418,-0.071681,-0.730634,-0.144538,-0.316398,-0.140504,0.568030,0.627514,0.342635,-0.513060,0.573039,...,0.690227,0.179910,0.819151,-0.986035,-0.217857,0.704062,-0.557054,0.336979,0.073345,0.416182,0.169119,0.252819,-0.007765,0.867754,0.049940,0.622713,0.249839,0.490119,0.017129,0.472541,-0.081607,0.270919,-0.168286,0.216581,-0.160095,1.078925,0.140712,0.485378,-0.825715,-0.538115,-0.143329,-0.438859,-0.328311,0.467558,-0.623120,-0.232646,0.089066,0.373773,0.051465,0.133306
1,-0.131595,0.717932,0.103539,-0.352256,0.445363,0.574004,0.224938,-0.443664,0.156914,-0.621618,0.611865,0.595117,0.254392,-0.395437,0.282696,-0.721986,0.495153,-0.142491,0.425335,-0.847287,0.158907,-0.121272,-0.077500,0.322465,0.245117,0.059716,0.712812,0.279434,-0.062502,-0.594381,-0.161564,-0.912918,-0.276822,0.038880,0.144402,0.864494,0.575056,0.140458,-0.946865,0.566941,...,0.402116,0.236811,0.438252,-0.977628,-0.213731,1.024718,-0.511329,0.466471,0.013676,-0.338417,0.193394,-0.484011,-0.175995,1.359603,0.611458,-0.022046,0.306897,0.529887,-0.013254,0.200802,-0.388851,-0.016475,-0.287569,0.173027,0.000487,0.696104,0.154334,0.583121,-0.620627,-0.281843,-0.357997,-0.563931,-0.109533,0.522604,-0.882951,-0.611184,-0.418232,0.575920,-0.099176,0.193249
2,-0.088683,0.253432,-0.127657,0.622538,-0.325812,0.536132,0.729754,-0.175889,-0.649913,-0.419049,-0.334742,0.199344,0.924215,-0.092979,-0.252962,0.571395,0.298841,-0.037249,0.201663,-0.605967,-0.471863,-0.401052,0.603825,0.514850,1.017304,-0.253662,0.632144,1.323611,-0.037013,-0.371983,-0.155334,-0.673828,-0.292144,-0.339262,0.097976,0.239314,0.449920,0.534731,-1.278293,0.211376,...,0.944642,0.162069,0.349109,-1.309283,-0.536939,0.179826,0.029545,0.720614,0.231857,-0.428832,-0.070528,-0.317812,-0.905764,0.546344,0.519607,0.382819,-0.553630,0.094056,0.145244,0.248824,-0.619912,-0.656022,0.469421,-0.663645,-0.103491,0.573311,-0.210589,0.457057,-0.521667,-0.271598,-0.267051,-0.230766,-0.338813,1.029949,-0.810742,-0.761941,-0.479853,0.077615,0.325930,-0.238579
3,-0.087107,0.653962,-0.185668,-0.346532,0.271266,0.130659,0.110633,-0.481047,-0.453030,-0.704346,0.269104,0.765400,-0.131103,-0.477464,0.208585,-0.560699,0.167637,-0.375447,0.031503,-0.373973,0.472703,0.225207,0.008906,0.703815,-0.013692,0.098833,0.512104,0.422383,-0.036169,-0.866069,-0.052607,-1.309379,0.249422,-0.075759,-0.154234,0.623039,0.071653,0.113011,-1.043562,0.201496,...,0.398728,0.430202,0.548670,-1.208655,-0.139523,1.113375,0.092824,0.219635,0.400243,-0.533676,0.536090,0.274127,-0.216970,0.898456,0.493920,0.672126,0.379411,0.108186,-0.388592,0.055922,-0.419779,-0.068736,0.082193,-0.113954,-0.304840,0.383442,0.005209,0.868236,-0.421514,-0.385263,-0.639858,-0.303500,0.164779,0.129109,-0.683032,-0.802295,-0.375369,0.249854,-0.136509,0.575346
4,-0.532874,0.162291,0.077257,-0.460296,0.101121,-0.125485,0.050201,-0.356446,-0.598464,-1.237478,0.488874,0.532229,-0.091280,-0.702574,0.335763,-0.642425,0.524485,-0.317519,0.089133,-0.480850,0.275057,-0.027158,0.426448,0.599703,0.038396,-0.081411,0.517921,0.242776,0.089475,-0.650904,0.042535,-0.911383,-0.222596,-0.157123,0.055192,0.288849,-0.067637,0.022072,-0.545285,0.417791,...,0.471688,0.172623,0.781899,-1.341941,-0.104758,1.206891,0.046972,0.373366,0.314850,0.170564,0.918850,0.559489,0.105526,0.945548,0.284143,0.772073,-0.208892,-0.008863,-0.276363,0.641212,-0.104338,0.394144,-0.004151,0.318418,-0.228820,-0.008893,-0.172069,0.425589,-0.102984,-0.597771,-0

In [140]:
df_meta = df_txt.loc[:, ['UNIT_TITLE', 'UNIT_CODE', 'HANDBOOK_SYNOPSIS', 'STUDY_LEVEL', 'OWNING_FACULTY', 'OWNING_ORG_UNIT']]

In [141]:
df_meta.head()

,UNIT_TITLE,UNIT_CODE,HANDBOOK_SYNOPSIS,STUDY_LEVEL,OWNING_FACULTY,OWNING_ORG_UNIT
0,Accounting in business,ACB1020,This unit introduces basic accounting concepts...,undergraduate,Faculty of Business and Economics,Department of Accounting
2,Financial accounting 1,ACB1120,This unit provides you with an introduction to...,undergraduate,Faculty of Business and Economics,Department of Accounting
5,Financial accounting 2,ACB2120,This unit provides an overview of the current ...,undergraduate,Faculty of Business and Economics,Department of Accounting
6,Management accounting 1,ACB2220,Introduction to management accounting. Topics ...,undergraduate,Faculty of Business and Economics,Department of Accounting
7,Accounting information systems,ACB2420,"The objective of this unit is two-fold. First,...",undergraduate,Faculty of Business and Economics,Department of Accounting


Remove newlines

In [145]:
df_meta['HANDBOOK_SYNOPSIS'] = df_meta.HANDBOOK_SYNOPSIS.apply(lambda x: x.replace("\n", " "))

### Export results

In [111]:
df_txt.to_csv("unit_synopsis_embeddings.csv", index=False)

In [130]:
df_embeddings.to_csv("unit_synopsis_embeddings_only.csv", sep="\t", header=False, index=False)

In [146]:
df_meta.to_csv("unit_synopsis_metadata_only.csv", sep="\t", index=False)

EOF